In [3]:
import pandas as pd
import data_util
from tec_an import TecAn
import numpy as np
from data_util import *
from bitstamp import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb


In [4]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = 1234,
                 val_end = 1234,
                 train_start_list = []
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [6]:
win = 40
minus = 15
step =  1

tec = TecAn(windows = win, windows_limit = 100)
sourceDataGenerator = SourceDataGenerator(tec = tec)
online = get_online_data(minus, sourceDataGenerator, True, win)

In [9]:
train_x_raw, train_y_raw = online.load_train_data()
train_x, train_y, _closed_prices = get_sequencial_data(train_x_raw, train_y_raw , step)

val_x_raw, val_y_raw = online.load_val_data("ltceur")
val_x, val_y, closed_prices = get_sequencial_data(val_x_raw, val_y_raw , step)


In [25]:
%%time
from catboost import CatBoostClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score


X_train, Y_train = train_x, train_y


clf = CatBoostClassifier()

params = {'iterations': [50, 500, 100],
          'depth': [
                    5,
                    6,
                    10
          ],
          'loss_function': ['Logloss', 'CrossEntropy'],
          'l2_leaf_reg': np.logspace(-20, -19, 3),
          'leaf_estimation_iterations': [10, 15, 20],
          #'eval_metric': ['recall'],
          #'use_best_model': ['True'],
          'logging_level':['Silent'],
          'random_seed': [42]
         }


CPU times: user 250 µs, sys: 13 µs, total: 263 µs
Wall time: 252 µs


In [26]:
%%time


clf_grid = gridSearch(X_train, Y_train, clf, params, make_scorer(recall_score))

#0.6743010531359825

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

KeyboardInterrupt: 

In [27]:
print(clf_grid.best_params_)

NameError: name 'clf_grid' is not defined

In [ ]:
#model = CatBoostClassifier(**clf_grid.best_params_)

#model.fit(X_train, Y_train)
#model.fit(x_norm, Y_train)

model = clf_grid.best_estimator_

eval_data(model, X_train, Y_train)

#5473554735547356

In [ ]:

eval_data(model, valX, valY)


In [17]:
from joblib import dump, load
dump(model, 'model/CatBoostClassifier') 

['model/CatBoostClassifier']

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

#model = clf_grid.best_estimator_

clf = Pipeline([
  ('feature_selection', SelectFromModel(
      LinearSVC()
  )
  ),
  ('classification', CatBoostClassifier(logging_level = 'Silent'))
])

clf.fit(X_train, Y_train)



Pipeline(steps=[('classification',
                 <catboost.core.CatBoostClassifier object at 0x7f94d4e69400>)])

In [28]:
eval_data(clf, X_train, Y_train)

eval_data(clf, valX, valY)

              precision    recall  f1-score   support

     class 0       0.88      0.91      0.89      2165
     class 1       0.90      0.86      0.88      1962

    accuracy                           0.89      4127
   macro avg       0.89      0.89      0.89      4127
weighted avg       0.89      0.89      0.89      4127

              precision    recall  f1-score   support

     class 0       0.49      0.58      0.54       219
     class 1       0.55      0.45      0.50       240

    accuracy                           0.52       459
   macro avg       0.52      0.52      0.52       459
weighted avg       0.52      0.52      0.51       459

